In [1]:
import os
import pandas as pd
import numpy as np
import pickle
import json
from argparse import ArgumentParser
from datetime import datetime
import sys
from sys import exit
from itertools import chain
import yaml
from pathlib import Path
from natsort import natsorted



import sklearn
from sklearn import metrics, ensemble
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.metrics import accuracy_score,confusion_matrix,ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split,cross_val_score,cross_val_predict,cross_validate,KFold
from joblib import parallel_backend

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import rdkit
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors

#! IF PERFORMING MULTIPLE RANDOM OPERATIONS, MAY ALSO NEED TO SPECIFY SEED IN OP ITSELF 
#! OTHERWISE, CHANGING ORDER OF OPS CHANGES RANDOM INITIALIZATION... BUT DOWNSIDE IS YOU ONLY HAVE 1 RANDOM SOURCE (possibly bad)
rs = 0
np.random.seed(rs)
sklearn.random.seed(rs)


In [2]:
data_path = Path.cwd().parent/"data"
csv_path = data_path/"csvs/"
pkl_path = data_path/"pkls/"
descriptor_path = csv_path/"descriptor_csvs/"

In [3]:
db_small_sol = pd.read_csv(csv_path/"SmallMoleculeSolDB_220607_exp_solu_DB_v2.csv")
sel_additives = ['2,4-dihydroxybenzophenone', 'dioctyl adipate',
       'decabromodiphenyl ether', 'melamine', 'stearic acid',
       'butyl stearate', 'stearyl alcohol', 'azodicarbonamide', 'sudan I']
print("Rows containing additives selected:",db_small_sol[db_small_sol.solute.isin(sel_additives)].shape[0])

Rows containing additives selected: 0


In [4]:
db_small_sol = db_small_sol.dropna(how='any', subset=["DGsolv"])
db_small_sol

,solute,solvent,logS,DGsolv,Reference (DOI),Comments,can_smiles_solute,can_smiles_solvent,concat_smiles,is_chonps_halogens,index
2,Br,CCCC,NaN,-1.23,10.1016/j.cej.2021.129307,['CompSol Binary (entry number=34224)'],Br,CCCC,Br.CCCC,True,3
3,Br,CCCCCC,NaN,-1.19,10.1016/j.cej.2021.129307,['CompSol Binary (entry number=45365)'],Br,CCCCCC,Br.CCCCCC,True,4
4,Br,CCCCCCCC,NaN,-1.11,10.1016/j.cej.2021.129307,['CompSol Binary (entry number=54328)'],Br,CCCCCCCC,Br.CCCCCCCC,True,5
5,Br,CCCCCCCCCC,NaN,-1.09,10.1016/j.cej.2021.129307,['CompSol Binary (entry number=37100)'],Br,CCCCCCCCCC,Br.CCCCCCCCCC,True,6
8,BrB(Br)Br,BrB(Br)Br,NaN,-4.72,Compsol pure,solute-solvent same,BrB(Br)Br,BrB(Br)Br,BrB(Br)Br.BrB(Br)Br,True,9
...,...,...,...,...,...,...,...,...,...,...,...
23587,c1cncnc1,c1cncnc1,NaN,-5.68,Compsol pure,solute-solvent same,c1cncnc1,c1cncnc1,c1cncnc1.c1cncnc1,True,23590
23589,o1cccn1,o1cccn1,NaN,-5.18,Compsol pure,solute-solvent same,c1cnoc1,c1cnoc1,c1cnoc1.c1cnoc1,True,23592
23592,o1ccnc1,o1ccnc1,NaN,-4.33,Compsol pure,solute-solvent same,c1cocn1,c1cocn1,c1cocn1.c1cocn1,True,23595
23595,N1N=CN=C1,CC(=O)OC,NaN,-10.33,10.1016/j.cej.2021.129307,"['Abraham Paper (A12)', 'Abraham Paper (A3)']",c1nc[nH]n1,COC(C)=O,c1nc[nH]n1.COC(C)=O,True,23598


### Generate Descriptors

In [5]:
def generate_rdmol_descriptors(descriptor_list, smiles_lst):
	"""From a descriptor list and a smiles list, returns a list and dataframe of rdMolDescriptors (with tuples handled appropriately)."""
	mol_list = [Chem.MolFromSmiles(x) for x in smiles_lst]

	rd_descriptors = []
	properties = rdMolDescriptors.Properties(descriptor_list)
	for i,entry in enumerate(mol_list):
		if entry is None or entry is np.nan:
			print("SMILES COULD NOT BE PARSED:",smiles_lst[i],entry,i)
			rd_descriptors.append('')
			continue
		else:
			properties_tuple = list(zip(properties.GetPropertyNames(), properties.ComputeProperties(entry)))
			rd_descriptors.append(properties_tuple)
	rd_descriptors = [list(x) for x in rd_descriptors]
	descriptor_df = pd.DataFrame(rd_descriptors, columns=descriptor_list)

	def get_second_elmt_tuple(x):
		if x is None:
			return np.nan
		else:
			return x[1]
	descriptor_df = descriptor_df.applymap(get_second_elmt_tuple)
	descriptor_df = descriptor_df.dropna()
	return descriptor_df




solute_properties_subset = ['lipinskiHBA', 
		'NumHBA', 
		'lipinskiHBD', 
		'NumHBD', 
		'NumRotatableBonds', 
		'NumHeteroatoms', 
		'NumAmideBonds', 
		'FractionCSP3', 
		'NumRings', 
		'NumAromaticRings', 
		'NumAliphaticRings', 
		'NumSaturatedRings', 
		'NumHeterocycles', 
		'NumSaturatedHeterocycles', 
		'NumAliphaticHeterocycles', 
		'NumAtomStereoCenters', 
		'tpsa',
		'chi0v',
		'chi1v',
		'chi2v',
		'chi3v',
		'chi4v',
		'kappa1',
		'kappa2',
		'kappa3']


solvent_properties_subset = ['lipinskiHBA', 
		'NumHBA', 
		'lipinskiHBD', 
		'NumHBD', 
		'NumRotatableBonds', 
		'NumHeteroatoms', 
		'NumAmideBonds', 
		'FractionCSP3', 
		'NumRings', 
		'NumAromaticRings', 
		'NumAliphaticRings', 
		'NumSaturatedRings', 
		'NumHeterocycles', 
		'NumSaturatedHeterocycles', 
		'NumAliphaticHeterocycles', 
		'tpsa',
		'chi0v',
		'chi1v',
		'chi2v',
		'chi3v',
		'chi4v',
		'kappa1',
		'kappa2',
		'kappa3']



#* Generate Descriptors: SOLUTE 
descrips_rdk_solute = generate_rdmol_descriptors(solute_properties_subset, \
														list(db_small_sol.can_smiles_solute))
#* Generate Descriptors: SOLVENTS
descrips_rdk_solv = generate_rdmol_descriptors(solvent_properties_subset, \
														list(db_small_sol.can_smiles_solvent))

#* Join Descriptors: Solute + Solvent
descrips_rdk_solute_solv = descrips_rdk_solute.join(descrips_rdk_solv,how='left',lsuffix='_solute', rsuffix='solvent')

[13:43:25] WARNING: not removing hydrogen atom without neighbors
[13:43:25] WARNING: not removing hydrogen atom without neighbors
[13:43:25] WARNING: not removing hydrogen atom without neighbors


Define train/test data for RF model

In [6]:
X = descrips_rdk_solute_solv
y = list(db_small_sol.DGsolv)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=rs)
X_train

,lipinskiHBA_solute,NumHBA_solute,lipinskiHBD_solute,NumHBD_solute,NumRotatableBonds_solute,NumHeteroatoms_solute,NumAmideBonds_solute,FractionCSP3_solute,NumRings_solute,NumAromaticRings_solute,...,NumAliphaticHeterocyclessolvent,tpsasolvent,chi0vsolvent,chi1vsolvent,chi2vsolvent,chi3vsolvent,chi4vsolvent,kappa1solvent,kappa2solvent,kappa3solvent
4861,0.0,0.0,0.0,0.0,6.0,0.0,0.0,1.00,0.0,0.0,...,0.0,26.30,4.023603,1.904030,0.347601,0.347601,0.203263,5.47,2.693714,3.470000
10043,2.0,2.0,0.0,0.0,0.0,2.0,0.0,0.00,0.0,0.0,...,0.0,29.46,5.390996,3.100685,0.877853,0.877853,0.413011,7.92,6.920000,5.920000
8596,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.00,1.0,1.0,...,0.0,20.23,6.396961,4.023335,1.511667,1.511667,0.892133,8.96,7.960000,7.960000
1427,4.0,4.0,0.0,0.0,6.0,4.0,0.0,0.20,0.0,0.0,...,0.0,52.60,7.770821,3.937524,1.042386,1.042386,0.514425,12.16,7.790843,6.844729
9591,8.0,5.0,1.0,1.0,3.0,8.0,0.0,0.00,1.0,1.0,...,0.0,26.30,4.023603,1.904030,0.347601,0.347601,0.203263,5.47,2.693714,3.470000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4859,0.0,0.0,0.0,0.0,6.0,0.0,0.0,1.00,0.0,0.0,...,0.0,20.23,2.154320,1.023335,0.000000,0.000000,0.000000,2.96,1.960000,1.960000
3264,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.75,0.0,0.0,...,0.0,20.31,3.432812,1.388328,0.210819,0.210819,0.000000,4.47,1.758184,3.470000
9845,3.0,2.0,2.0,2.0,1.0,3.0,0.0,0.00,1.0,1.0,...,0.0,20.23,3.568534,2.023335,0.511667,0.511667,0.158114,4.96,3.960000,3.960000
10799,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.00,0.0,0.0,...,0.0,50.09,3.970817,1.747210,0.451065,0.451065,0.136719,5.96,3.161613,2.771244


In [7]:
regr = sklearn.ensemble.RandomForestRegressor(random_state=rs)

with parallel_backend('threading', n_jobs=10): 
    fit_model = regr.fit(X_train, y_train)
fit_model

RandomForestRegressor(random_state=0)

In [8]:
import pickle
with open(data_path/"pkls/fit_RF_model_smallmol.pkl", "wb") as f:
    pickle.dump(fit_model, f)